In [2]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install -q kaggle
!pip install -q tensorflow[and-cuda]
!pip install -q keras-tuner
!pip install -q optuna
!pip install -q numba
!pip install -q pyspark
!pip install -q termcolor
!pip install -q seaborn
!pip install -q matplotlib
!pip install -q pandas
!pip install -q numpy
!pip install -q scikit-learn
!pip install -q optuna-integration
!pip install -q numba
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    "cudf-cu12==24.10.*" "dask-cudf-cu12==24.10.*" "cuml-cu12==24.10.*" \
    "cugraph-cu12==24.10.*" "nx-cugraph-cu12==24.10.*" "cuspatial-cu12==24.10.*" \
    "cuproj-cu12==24.10.*" "cuxfilter-cu12==24.10.*" "cucim-cu12==24.10.*" \
    "pylibraft-cu12==24.10.*" "raft-dask-cu12==24.10.*" "cuvs-cu12==24.10.*" \
    "nx-cugraph-cu12==24.10.*"

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import os
import subprocess
import numpy as np
import pandas as pd
import gc
import ctypes
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow and Keras for deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K, Model, Input, layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv1D, BatchNormalization, ReLU, MaxPooling1D, GlobalAveragePooling1D, Add, Concatenate, Input, LayerNormalization, MultiHeadAttention, Flatten, Reshape, LeakyReLU, Activation, Lambda, Layer
from tensorflow.keras.backend import clear_session
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError, Metric
from tensorflow.keras.optimizers import Adam, RMSprop, Adamax
from tensorflow.keras.callbacks import EarlyStopping, Callback, LearningRateScheduler
from tensorflow.keras.mixed_precision import Policy, set_global_policy
from tensorflow.keras.regularizers import l2, l1_l2
from tensorflow.data import Dataset
from keras import mixed_precision

# Scikit-learn for machine learning utilities
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler as sklearn_StandardScaler

# PySpark for distributed data processing
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, monotonically_increasing_id
from pyspark.sql.types import ArrayType, FloatType, IntegerType, DoubleType, StringType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler as SparkVectorAssembler, MinMaxScaler as SparkMinMaxScaler, RobustScaler, StandardScaler as SparkStandardScaler, PCA as SparkPCA
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, DenseVector
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Keras Tuner for hyperparameter tuning
import keras_tuner as kt
from keras_tuner import HyperParameters, Hyperband

# Optuna for hyperparameter optimization
import optuna
from optuna.integration import TFKerasPruningCallback

# Numba for CUDA GPU acceleration
from numba import cuda

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Logging information for dynamic error handling
import logging

# Console output coloring
from termcolor import colored

# Import sqrt function from math library
from math import sqrt

# Import count function from itertools library
from itertools import count
# Import mean, stddev, col, when functions from pyspark.sql.functions library
from pyspark.sql.functions import mean, stddev, col, when

2024-11-16 02:03:13.846014: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 02:03:13.853044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731718993.861043   66153 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731718993.863486   66153 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 02:03:13.872244: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# 🎮 *NVIDIA GPU GUIDE FOR KAGGLE* 🚀
## 🌟 A Magical Journey Through GPU Computing 🌟
![NVIDIA RTX](https://img.shields.io/badge/NVIDIA-RTX-76B900?style=for-the-badge&logo=nvidia&logoColor=white)

## 🎯 Choose Your Path:

### 1. 🎮 The Gamer's Quest
- Level up from gaming to ML/Data Science mastery
- Harness your GPU's hidden potential

### 2. 🤖 The AI Engineer's Saga 
- From development to production deployment
- Craft powerful AI solutions

### 3. ⚡ The HPC Wizard's Chronicle
- Master the art of High Performance Computing
- Unlock unprecedented computational power

### 4. 🔥 The Enthusiast's Adventure
- Push boundaries with extreme overclocking
- Pioneer cutting-edge AI research

![GPU Architecture](https://img.shields.io/badge/CUDA-Powered-76B900?style=for-the-badge&logo=nvidia&logoColor=white)

In [5]:
# Initialize Spark session for the master node
# 1 - Use all available local cores for the master node
# 2 - Set the application name for the master node
# 3 - Configure the driver and executor memory and GPU settings
import subprocess
import json

# Configure Spark with GPU settings for RTX 4090 (24GB) and A5000 (24GB)
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ERROR 418 - I'M A TEAPOT") \
    .config("spark.driver.memory", "50g") \
    .config("spark.executor.memory", "50g") \
    .config("spark.driver.maxResultSize", "50g") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.rapids.memory.pinnedPool.size", "24G") \
    .config("spark.rapids.sql.concurrentGpuTasks", "2") \
    .config("spark.rapids.memory.gpu.pooling.enabled", "true") \
    .config("spark.rapids.memory.gpu.allocFraction", "0.95") \
    .config("spark.rapids.sql.explain", "ALL") \
    .config("spark.executor.resource.gpu.amount", "2") \
    .config("spark.task.resource.gpu.amount", "0.25") \
    .config("spark.rapids.sql.incompatibleOps.enabled", "true") \
    .config("spark.rapids.memory.host.spillStorageSize", "64G") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
    .config("spark.rapids.memory.gpu.maxAllocFraction", "0.95") \
    .config("spark.rapids.sql.batchSizeBytes", "512M") \
    .config("spark.rapids.sql.reader.batchSizeRows", "100000") \
    .config("spark.rapids.sql.variableRowGroupSize.enabled", "true") \
    .getOrCreate()

gpus = tf.config.list_logical_devices('GPU')
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
      a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
      b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
      c.append(tf.matmul(a, b))

  with tf.device('/CPU:0'):
    matmul_sum = tf.add_n(c)

  print(matmul_sum)

# Set log level to INFO
spark.sparkContext.setLogLevel("INFO")
# Configure logging
log4jLogger = spark._jvm.org.apache.log4j
logger = log4jLogger.LogManager.getLogger(__name__)

def custom_logger(level, message):
    color = 'white'
    if level == "INFO":
        color = 'cyan'
    elif level == "SUCCESS":
        color = 'green'
    elif level == "ERROR":
        color = 'red'
    elif level == "ACTION":
        color = 'blue'
    elif level == "PROGRESS" or level == "WARNING":
        color = 'yellow'
    elif level == "FINAL":
        color = 'magenta'
    logger.info(colored(f"SPARK: {level} - {message}", color))

# Set the environment variable for memory allocator
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

def initialize_cuda():
    cuda = ctypes.CDLL('libcuda.so')
    cuInit = cuda.cuInit
    cuInit.restype = ctypes.c_int
    cuInit.argtypes = [ctypes.c_uint]
    cuDeviceGetCount = cuda.cuDeviceGetCount
    cuDeviceGetCount.restype = ctypes.c_int
    cuDeviceGetCount.argtypes = [ctypes.POINTER(ctypes.c_int)]
    cuDeviceGet = cuda.cuDeviceGet
    cuDeviceGet.restype = ctypes.c_int
    cuDeviceGet.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_int]

    res = cuInit(0)
    if res != 0:
        raise RuntimeError("Failed to initialize CUDA")
    device_count = ctypes.c_int()
    res = cuDeviceGetCount(ctypes.byref(device_count))
    if res != 0:
        raise RuntimeError("Failed to get device count")
    devices = []
    for i in range(device_count.value):
        device = ctypes.c_int()
        res = cuDeviceGet(ctypes.byref(device), i)
        if res != 0:
            raise RuntimeError(f"Failed to get device {i}")
        devices.append(device.value)
    return devices

def allocate_gpu_memory(size):
    cuda = ctypes.CDLL('libcuda.so')
    cuda_malloc = cuda.cuMemAlloc
    cuda_malloc.restype = ctypes.c_int
    cuda_malloc.argtypes = [ctypes.POINTER(ctypes.c_ulonglong), ctypes.c_ulonglong]
    ptr = ctypes.c_ulonglong()
    res = cuda_malloc(ctypes.byref(ptr), size)
    if res != 0:
        raise RuntimeError("Failed to allocate GPU memory")
    return ptr.value

def free_gpu_memory(ptr):
    cuda = ctypes.CDLL('libcuda.so')
    cuda_free = cuda.cuMemFree
    cuda_free.restype = ctypes.c_int
    cuda_free.argtypes = [ctypes.c_ulonglong]
    res = cuda_free(ptr)
    if res != 0:
        raise RuntimeError("Failed to free GPU memory")

# Check GPU memory usage
def check_gpu_memory():
    os.system('nvidia-smi')

# Initialize CUDA
cuda_devices = initialize_cuda()

# Check available GPU memory
check_gpu_memory()

# Ensure TensorFlow uses the GPU by setting memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")

# Check and print the number of GPUs available
num_gpus = len(tf.config.list_physical_devices('GPU'))
print(f"Num GPUs Available: {num_gpus}")

# Perform a simple TensorFlow operation to verify GPU usage
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], dtype=tf.float32)
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype=tf.float32)
    c = tf.matmul(a, b)
print(f"Result of matrix multiplication on GPU:\n{c.numpy()}")

def get_gpu_memory_usage():
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        memory_used = int(result.stdout.decode('utf-8').strip())
        return memory_used
    except Exception as e:
        print(f"Failed to get GPU memory usage: {e}")
        return None

def freegpu():
    try:
        # Get GPU memory usage before freeing
        memory_before = get_gpu_memory_usage()

        result = subprocess.run(['sudo', 'fuser', '-v', '/dev/nvidia*'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for line in result.stdout.decode('utf-8').split('\n'):
            if '/dev/nvidia' in line:
                pid = int(line.split()[-1])
                os.kill(pid, 9)

        # Get GPU memory usage after freeing
        memory_after = get_gpu_memory_usage()

        if memory_before is not None and memory_after is not None:
            memory_freed = memory_before - memory_after
            print(f"GPU memory has been freed. {memory_freed} MB of GPU memory was released.")
        else:
            print("GPU memory has been freed.")
    except Exception as e:
        print(f"Failed to free GPU memory: {e}")

# Set the environment variable for XLA flags
os.environ['XLA_FLAGS'] = '--xla_gpu_strict_conv_algorithm_picker=false'

# ======================================================================================================
# ==============================[SETUP MIRROREDSTRATEGY WITHOUT NCCL]===================================
# ======================================================================================================

# ------------------------------------------------------------------------------------------------------
# Define the strategy with HierarchicalCopyAllReduce to avoid NCCL
# ------------------------------------------------------------------------------------------------------
strategy = tf.distribute.MirroredStrategy(
    devices=["/gpu:0", "/gpu:1"],
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce()
)

print(f"Number of devices under strategy: {strategy.num_replicas_in_sync}")

24/11/16 02:03:15 WARN Utils: Your hostname, master resolves to a loopback address: 127.0.1.1; using 192.168.1.125 instead (on interface eno2)
24/11/16 02:03:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 02:03:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/16 02:03:16 WARN ResourceUtils: The configuration of cores (exec = 32 task = 1, runnable tasks = 32) will result in wasted resources due to resource gpu limiting the number of runnable tasks per executor to: 8. Please adjust your configuration.
I0000 00:00:1731718996.891799   66153 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20525 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:02:00.0, compute capability: 8.9
I0000 00:

tf.Tensor(
[[ 44.  56.]
 [ 98. 128.]], shape=(2, 2), dtype=float32)
Sat Nov 16 02:03:17 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:01:00.0  On |                    0 |
| 41%   63C    P2             85W /  230W |   20974MiB /  23028MiB |     14%      Default |
|                                         |                        |    

### **LazyEval EDA**

In [6]:
# Load the processed data for autoencoder training
try:
    # Try Kaggle path first
    original_data = pd.read_csv('/kaggle/input/nvidia-complete-lineup-dataset/Updated_GPU_Dataset_with_Tensor_Core_Information.csv', index_col=False, header=0)
except FileNotFoundError:
    # If Kaggle path fails, try local path
    try:
        original_data = pd.read_csv('/home/skander/datasets/gpu/Updated_GPU_Dataset_with_Tensor_Core_Information copy.csv', index_col=False, header=0)
    except FileNotFoundError:
        print("Both Kaggle and local paths failed. Please check the file paths and try again.")

custom_logger("INFO", "ORIGINAL DATA - BEFORE PROCESSING")

# Univariate analysis
def univariate_analysis(df):
    print("Univariate Analysis\n" + "="*40)
    
    # Shape
    print(f"Shape:\n{df.shape}\n")
    
    # Data Types
    print(f"Data Types:\n{df.dtypes}\n")
    
    # Missing Values
    missing_values = df.isnull().sum()
    print(f"Missing Values:\n{missing_values}\n")
    
    # Duplicates
    print(f"Duplicates:\n{df.duplicated().sum()}\n")
    
    # Summary statistics for numerical columns
    print("Summary Statistics for Numerical Columns:\n")
    print(df.describe(include=[np.number]))
    
    # Detailed information about NaN values
    print("\nDetailed Information about NaN Values:\n")
    for col in df.columns:
        if df[col].isnull().any():
            print(f"Column '{col}' has {df[col].isnull().sum()} NaN values, which is {df[col].isnull().mean() * 100:.2f}% of the column.")

# Perform univariate analysis
univariate_analysis(original_data)


Univariate Analysis
Shape:
(107, 40)

Data Types:
Model                            object
Micro-Architecture / Process     object
Launch                           object
Cuda Cores                       object
Base Clock (MHz)                 object
Max Boost Clock (MHz)            object
Bus Type                         object
Bus Width (bits)                 object
Size (GB)                         int64
Clock (MT/s)                     object
Bandwidth (GB/s)                 object
Half Precision (GFLOPS)          object
Single Precision (GFLOPS)        object
Double Precision (GFLOPS)        object
TDP (Watts)                       int64
 Price                           object
 Scaled Performance HP           object
 Scaled Performance SP           object
 Scaled Performance DP           object
P/P HP (GFLOPS/$)               float64
P/P SP (GFLOPS/$)               float64
P/P DP (GFLOPS/$)               float64
Scaled P/P HP (%)                object
Scaled P/P SP (%)             

24/11/16 02:03:17 INFO __main__: SPARK: INFO - ORIGINAL DATA - BEFORE PROCESSING


### **Lack of features - adding Tensor Cores, Generation, and End-of-Life**

In [7]:
import pandas as pd

def clean_duplicates(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check for duplicates based on Model column
    duplicates = df[df.duplicated(subset=['Model'], keep=False)]
    print(f"Found {len(duplicates)} duplicate entries")
    
    # Remove duplicates keeping first occurrence
    df_cleaned = df.drop_duplicates(subset=['Model'], keep='first')
    
    # Save cleaned dataset
    output_path = file_path.replace('.csv', '_cleaned.csv')
    df_cleaned.to_csv(output_path, index=False)
    
    return df_cleaned, duplicates

# Usage
file_path = '/home/skander/datasets/gpu/Updated_GPU_Dataset_with_Tensor_Core_Information copy.csv'
df_cleaned, duplicates = clean_duplicates(file_path)

# Print duplicate models for verification
print("\nDuplicate Models found:")
for model in duplicates['Model'].unique():
    print(f"- {model}")

Found 82 duplicate entries

Duplicate Models found:
- Tesla H100
- GeForce RTX 4080
- RTX 4500 Ada Generation
- Tesla A100 (80 GB)
- Tesla A16
- RTX A6000
- GeForce RTX 3090 Ti
- Tesla A40
- Tesla L4
- GeForce RTX 3090
- Tesla A100 (40 GB)
- RTX 4000 Ada Generation
- RTX A5500 
- GeForce RTX 4060 Ti (16 GB)
- Tesla A10
- RTX A5000
- RTX 4000 SFF Ada Generation
- Tesla A30
- RTX A4500 
- Tesla V100 (32 GB)
- Tesla V100 (16 GB)
- Nvidia TITAN RTX
- RTX A4000 
- Tesla P40
- Tesla T4
- Tesla P100 (16 GB)
- Tesla M60
- Tesla M40 (24 GB)
- Tesla K80
- Tesla M10
- Tesla A2
- GeForce RTX 4070 Ti
- GeForce RTX 3080 Ti
- GeForce RTX 3080 (12 GB)
- GeForce RTX 4070
- GeForce RTX 2060 (12 GB)
- GeForce RTX 3060 (12 GB)
- RTX A2000 (12 GB)
- Tesla P100 (12 GB)
- Tesla M40 (12 GB)
- Tesla K40


#### **Duplicate Elimination**

In [9]:
def validate_cleaned_data(df_original, df_cleaned):
    # Verify no unique information was lost
    for model in df_original['Model'].unique():
        original_rows = df_original[df_original['Model'] == model]
        cleaned_row = df_cleaned[df_cleaned['Model'] == model]
        
        if len(original_rows) > 1:
            # Compare key metrics between duplicates
            metrics = ['Cuda Cores', 'Tensor Cores', 'Half Precision (GFLOPS)', 
                      'Single Precision (GFLOPS)', 'Double Precision (GFLOPS)']
            
            for metric in metrics:
                if original_rows[metric].nunique() > 1:
                    print(f"Warning: Different {metric} values found for {model}")
                    print(f"Values: {original_rows[metric].unique()}")

    # Verify counts
    print(f"\nOriginal dataset: {len(df_original)} rows")
    print(f"Cleaned dataset: {len(df_cleaned)} rows")
    print(f"Removed {len(df_original) - len(df_cleaned)} duplicate entries")

# Additional data quality checks
def check_data_quality(df):
    # Check for inconsistent tensor core generations
    arch_gen_map = {
        'Ada Lovelace': '4th Gen (Ada)',
        'Hopper': '4th Gen (Hopper)',
        'Ampere': '3rd Gen (Ampere)',
        'Turing': '2nd Gen (Turing)',
        'Volta': '1st Gen (Volta)'
    }
    
    for idx, row in df.iterrows():
        arch = row['Micro-Architecture / Process']
        gen = row['Tensor Core Gen']
        
        for arch_name, expected_gen in arch_gen_map.items():
            if arch_name in str(arch) and gen != expected_gen and gen != 'No Tensor Cores':
                print(f"Warning: Inconsistent tensor core generation for {row['Model']}")
                print(f"Architecture: {arch}, Listed Generation: {gen}")

# Run the validation
df_original = pd.read_csv('/home/skander/datasets/gpu/Updated_GPU_Dataset_with_Tensor_Core_Information copy.csv')
df_cleaned = pd.read_csv('/home/skander/datasets/gpu/Updated_GPU_Dataset_with_Tensor_Core_Information copy_cleaned.csv')

validate_cleaned_data(df_original, df_cleaned)
check_data_quality(df_cleaned)


Original dataset: 107 rows
Cleaned dataset: 66 rows
Removed 41 duplicate entries
